In [1]:
%%writefile mapper.py

import sys
import re
from collections import Counter

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

with open('stop_words_en.txt', 'r') as f:
    stop_words = [s.strip() for s in f.readlines()]
        
for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    
    all_words_cnt = 0
    stop_words_cnt = 0
    c = Counter()
    
    for word in words:
        all_words_cnt += 1
        if word in stop_words:
            c.update({word: 1})
        
    for word in c:
        key = word
        count = c[key]
        stop_words_cnt += count
    
    print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % all_words_cnt
    print >> sys.stderr, "reporter:counter:Wiki stats,Stop words,%d" % stop_words_cnt

    print("%d\t%d" % (stop_words_cnt, all_words_cnt))

Writing mapper.py


In [2]:
%%writefile reducer.py

import sys

current_key = None
word_sum = 0

for line in sys.stdin:
    try:
        stop_words_cnt, all_words_cnt = line.strip().split('\t', 1)
        stop_words_cnt = int(stop_words_cnt)
        all_words_cnt = int(all_words_cnt)
    except ValueError as e:
        continue

    print "%d\t%d" % (stop_words_cnt, all_words_cnt)

Writing reducer.py


In [3]:
%%bash

OUT_DIR="wordcount_result_"$(date +"%s%6N")
OUT_LOG="result.txt"
NUM_REDUCERS=8

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py,/datasets/stop_words_en.txt \
    -mapper "python mapper.py" \
    -combiner "python reducer.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null 2> ${OUT_LOG}

cat ${OUT_LOG} >&2
grep ' words=' ${OUT_LOG} \
    | cut -d'=' -f2 \
    | paste -sd' ' \
    | awk '{ printf "%f", ($1 / $2 * 100) }'

38.440698

rm: `wordcount_result_1514047728190417': No such file or directory
17/12/23 16:48:51 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/12/23 16:48:51 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/12/23 16:48:52 INFO mapred.FileInputFormat: Total input files to process : 1
17/12/23 16:48:52 INFO mapreduce.JobSubmitter: number of splits:2
17/12/23 16:48:53 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1514023825782_0001
17/12/23 16:48:53 INFO impl.YarnClientImpl: Submitted application application_1514023825782_0001
17/12/23 16:48:53 INFO mapreduce.Job: The url to track the job: http://79416c364193:8088/proxy/application_1514023825782_0001/
17/12/23 16:48:53 INFO mapreduce.Job: Running job: job_1514023825782_0001
17/12/23 16:49:00 INFO mapreduce.Job: Job job_1514023825782_0001 running in uber mode : false
17/12/23 16:49:00 INFO mapreduce.Job:  map 0% reduce 0%
17/12/23 16:49:16 INFO mapreduce.Job:  map 5% reduce 0%
17/12/23 16:49